## The first step is setting up the IBC light clients that live on each counterpart's chain. We will heavily utilize terra.py (https://github.com/terra-money/terra.py) and terra_proto (https://github.com/terra-money/terra.proto) in this series, but you can absolutely use more generalized packages to interact with lcd endpoints and generate proto classes.

## Additionally, we use Setten (https://setten.io/) for private terra rpc endpoints; Osmosis provides a public rpc that we can use.

In [ ]:
#imports

import pandas as pd
import json
import os
import sys
import base64
import requests
import subprocess
import math
import hashlib
import bech32
import time

from dateutil.parser import parse
from datetime import datetime, timedelta
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_decode, bech32_encode, convertbits

from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.wasm import MsgStoreCode, MsgInstantiateContract, MsgExecuteContract
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.bech32 import get_bech
from terra_sdk.core import AccAddress, Coin, Coins
from terra_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.core.wasm.data import AccessConfig
from terra_sdk.client.lcd.api._base import BaseAsyncAPI, sync_bind

from terra_proto.cosmwasm.wasm.v1 import AccessType
from terra_proto.cosmos.tx.v1beta1 import Tx, TxBody, AuthInfo, SignDoc, SignerInfo, ModeInfo, ModeInfoSingle, BroadcastTxResponse
from terra_proto.cosmos.base.abci.v1beta1 import TxResponse
from terra_proto.cosmos.tx.signing.v1beta1 import SignMode
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.types import ValidatorSet, Validator, SignedHeader, Header as tendermintHeader, Commit, BlockId, PartSetHeader, CommitSig, BlockIdFlag
from terra_proto.tendermint.version import Consensus
from terra_proto.tendermint.crypto import PublicKey
from betterproto.lib.google.protobuf import Any
from betterproto import Timestamp

In [ ]:
#misc helper functions
sys.path.append(os.path.join(os.path.dirname(__name__), '..', 'scripts'))

from helpers import proto_to_binary, timestamp_string_to_proto, stargate_msg, create_ibc_client

## We can actually use terra's lcd client & wallet classes for pretty much any cosmos chain.

In [ ]:
#lcd setup
class BaseAsyncAPI2(BaseAsyncAPI):
    async def query(self, query_string: str, params=None):
        if params is None:
          res = await self._c._get(query_string)
        else:
          res = await self._c._get(query_string, params=params)
        return res

    #for dispatching protobuf classes to the chain
    async def broadcast(self, tx):
        res = await self._c._post("/cosmos/tx/v1beta1/txs", {"tx_bytes": proto_to_binary(tx), "mode": "BROADCAST_MODE_BLOCK"})
        return res


class BaseAPI2(BaseAsyncAPI2):
    @sync_bind(BaseAsyncAPI2.query)
    def query(self, query_string: str):
        pass

    @sync_bind(BaseAsyncAPI2.broadcast)
    def broadcast(self, tx: Tx):
        pass

#terra
terra = LCDClient(url="https://pisco-lcd.terra.dev/", chain_id="pisco-1")
terra.broadcaster = BaseAPI2(terra)

#osmosis
osmo = LCDClient(url="https://lcd-test.osmosis.zone", chain_id="localterra")
osmo.chain_id = "osmo-test-4"
osmo.broadcaster = BaseAPI2(osmo)

In [ ]:
#rpc setup

#terra
setten_project_id = "a0a3abea69544a99a67700ce2c7926fb"
setten_project_key = "4a68cbb2303d4c109ea99f7bf7ede000"
terra_rpc_url = f"https://rpc.pisco.terra.setten.io/{setten_project_id}"
terra_rpc_header = {"Authorization": f"Bearer {setten_project_key}"}


#osmosis
osmo_rpc_url = "https://rpc-test.osmosis.zone"

In [ ]:
#wallet setup

#terra
wallet = terra.wallet(MnemonicKey(mnemonic="differ flight humble cry abandon inherit noodle blood sister potato there denial woman sword divide funny trash empty novel odor churn grid easy pelican"))

#osmosis
class OsmoKey(MnemonicKey):
  @property
  def acc_address(self) -> AccAddress: 
    if not self.raw_address:
      raise ValueError("could not compute acc_address: missing raw_address")
    return AccAddress(get_bech("osmo", self.raw_address.hex()))

osmo_wallet = osmo.wallet(OsmoKey(mnemonic="differ flight humble cry abandon inherit noodle blood sister potato there denial woman sword divide funny trash empty novel odor churn grid easy pelican", coin_type=118))

## Terra provides the python protobuf classes (via betterproto) for ibc/tendermint; again other python-ized protos may be used (or self-generated)

In [ ]:
#terra_proto imports
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.version import Consensus


## Create ibc light clients by submitting stargate messages containing context data from the counterpart chain

In [ ]:
#fetch osmosis tendermint data
unbonding_period = int(osmo.staking.parameters()["unbonding_time"].replace('s', ''))
trusting_period = math.floor(unbonding_period * 2 / 3)
max_clock_drift = 20
latest_height_revision_number = 4
tendermint_info = osmo.tendermint.block_info()["block"]

print(f"""
osmosis information for client on terra

unbonding_period: {unbonding_period}
trusting_period: {trusting_period}
max_clock_drift: {max_clock_drift}
tendermint_info: {tendermint_info}
""")

#create the ibc client on terra, containing fetched osmosis tendermint data
msg = MsgCreateClient(
client_state=Any(
  type_url="/ibc.lightclients.tendermint.v1.ClientState",
  value=ClientState(
    chain_id=osmo.chain_id,
    trust_level=Fraction(1,3),
    trusting_period=timedelta(seconds=trusting_period),
    unbonding_period=timedelta(seconds=unbonding_period),
    max_clock_drift=timedelta(seconds=max_clock_drift),
    frozen_height=Height(0,0),
    latest_height=Height(latest_height_revision_number, int(tendermint_info["header"]["height"])),
    proof_specs=[
      ProofSpec(
        leaf_spec=LeafOp(
          hash=HashOp.SHA256,
          prehash_key=HashOp.NO_HASH,
          prehash_value=HashOp.SHA256,
          length=LengthOp.VAR_PROTO,
          prefix=base64.b64decode(b"AA=="),
        ),
        inner_spec=InnerSpec(
          child_order=[0,1],
          child_size=33,
          min_prefix_length=4,
          max_prefix_length=12,
          #empty_child=b'',
          hash=HashOp.SHA256,
        ),
        max_depth=0,
        min_depth=0
      ),
      ProofSpec(
        leaf_spec=LeafOp(
          hash=HashOp.SHA256,
          prehash_key=HashOp.NO_HASH,
          prehash_value=HashOp.SHA256,
          length=LengthOp.VAR_PROTO,
          prefix=base64.b64decode(b"AA=="),
        ),
        inner_spec=InnerSpec(
          child_order=[0,1],
          child_size=32,
          min_prefix_length=1,
          max_prefix_length=1,
          #empty_child=b'',
          hash=HashOp.SHA256,
        ),
        max_depth=0,
        min_depth=0
      ),
    ],
    upgrade_path=["upgrade", "upgradedIBCState"],
    allow_update_after_expiry=True,
    allow_update_after_misbehaviour=True,
  ).SerializeToString()
),
consensus_state=Any(
  type_url="/ibc.lightclients.tendermint.v1.ConsensusState",
  value=ConsensusState(
    timestamp=timestamp_string_to_proto(tendermint_info["header"]["time"]),
    root=MerkleRoot(base64.b64decode(tendermint_info["header"]["app_hash"])),
    next_validators_hash=base64.b64decode(tendermint_info["header"]["next_validators_hash"]),
  ).SerializeToString(),
),
signer=wallet.key.acc_address,
)

print(f"""

ibc client creation message to be run on terra

{msg.to_dict()}

""")

#dispatch the message
result = stargate_msg("/ibc.core.client.v1.MsgCreateClient", msg, wallet, terra)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
client_id_on_terra = result_df[result_df["key"]=="client_id"]["value"].values[0]

print(f"""

client_id on terra: {client_id_on_terra}

""")

In [ ]:
#use helper function, create_ibc_client, to fetch terra tendermint data, fabricate the MsgCreateClient msg, and dispatch to osmosis chain
#ie. execute same steps as previous cell, but with reversed roles (terra consensus info on osmo chain)

result = create_ibc_client(terra, osmo, osmo_wallet, 1)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
client_id_on_osmo = result_df[result_df["key"]=="client_id"]["value"].values[0]

print(f"""

client_id on osmo: {client_id_on_osmo}

""")

## The key results from this step are the client_id's built each on terra and osmosis. Let's take a look at the client_id's, and the underlying client state data stored onchain.

In [ ]:
#client_id on terra, and its underlying state representing a snapshot summary of osmosis's consensus/tendermint state
client_state_on_terra = terra.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_terra}")

print(f"""

client_id on terra: {client_id_on_terra}
client_state on terra: {client_state_on_terra}

""")

In [ ]:
#client_id on osmosis, and its underlying state representing a snapshot summary of terra's consensus/tendermint state
client_state_on_osmo = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")

print(f"""

client_id on osmosis: {client_id_on_osmo}
client_state on osmosis: {client_state_on_osmo}

""")

## Persist client_id's for next steps.

In [ ]:
#save to current directory

data = {
    "client_id_on_terra": client_id_on_terra,
    "client_id_on_osmo": client_id_on_osmo,
}

with open("context.json", "w") as f:
    # Write the dictionary to the file as a JSON string
    json.dump(data, f)


## And that's it for IBC client setup! Please continue to the next notebook to create an ibc connection between terra and osmosis.